# Navier Stokes

## Introduction
We ignore any external forces and assume that the fluid is incompressible.

Given:

- $\vec{v}$: velocity vector field with x / y components $u$ and $v$
- $p$: pressure vector field
- $\rho$: density
- $\nu$: kinematic viscosity

And:

- $\nabla$ is the gradient operator
- $\nabla^2$ or $\Delta$ is the Laplacian operator


The equations for the incompressible Navier Stokes equations are:

1. Momentum conservation:
$$\frac{\partial \mathbf{u}}{\partial t} + \vec{v} \cdot \nabla \vec{v} = -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v}$$

2. Incompressibility or mass conservation:
$$\nabla \cdot \vec{v} = 0$$

We have the followin equivalence:

$$ \nabla \cdot \vec{v} = \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y}$$

And coresspondingly:

$$ \vec{v} \nabla \cdot \vec{v} = u \frac{\partial u}{\partial x} + v \frac{\partial v}{\partial y}$$


We can split the momentum conservation equation into two equations, one for each component of the velocity vector field:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right) \tag{1}$$

$$\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right) \tag{2}$$

Furthermore we can take the divergence of the momentum conservation equation and use the incompressibility equation to get a Poisson equation for the pressure vector field:

$$\nabla \left(\frac{\partial \mathbf{u}}{\partial t} + \vec{v} \cdot \nabla \vec{v}\right) = \nabla \left( -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v}  \right)$$

On the LHS the term $\nabla \cdot \vec{v}$ is zero, so we get:


$$\nabla \left(\vec{v} \cdot \nabla \vec{v}\right) = \nabla \left( -\frac{1}{\rho} \nabla p + \nu \Delta \vec{v} \right)$$

On the RHS $\nabla \nu \nabla^2 \vec{v}$ is zero, so we get:

$$\nabla \left(\vec{v} \cdot \nabla \vec{v}\right) = \nabla \left( -\frac{1}{\rho} \nabla p \right) = -\frac{1}{\rho} \Delta p$$

Which we can rearange to:

$$\Delta p = -\rho \nabla \left(\vec{v} \cdot \nabla \vec{v}\right)$$

Which can be written in 2D as:

$$\frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2} = -\rho \left(  \frac{\partial u}{\partial x} \frac{\partial u}{\partial x} + 2 \frac{\partial u}{\partial y} \frac{\partial v}{\partial x} + \frac{\partial v}{\partial y} \frac{\partial v}{\partial y} \right) \tag{3}$$

This equation couples the pressure and velocity fields. This is only necessary in the case of incompressible fluids. For compressible fluids the pressure is a function of the density and the momentum conservation equation is:

## Discretization

Lets discretize the u momentum conservation equation (1):

LHS: 
$$\frac{u^{n+1}_{i,j} - u^n_{i,j}}{\Delta t} + u^n_{i,j} \frac{u^n_{i,j} - u^n_{i-1,j}}{\Delta x} + v^n_{i,j} \frac{u^n_{i,j} - u^n_{i,j-1}}{\Delta y}$$

RHS:
$$ -\frac{1}{\rho} \frac{p^{n+1}_{i,j} - p^{n+1}_{i-1,j}}{ 2\Delta x} + \nu \left( \frac{u^n_{i+1,j} - 2 u^n_{i,j} + u^n_{i-1,j}}{\Delta x^2} + \frac{u^n_{i,j+1} - 2 u^n_{i,j} + u^n_{i,j-1}}{\Delta y^2} \right)$$

Sources: 

- https://digitalcommons.georgiasouthern.edu/cgi/viewcontent.cgi?article=1846
- https://nbviewer.org/github/barbagroup/CFDPython/blob/master/lessons/12_Step_9.ipynb
